In [18]:
from elasticsearch import Elasticsearch
import wikipedia

from typing import Any, List
from decouple import config
from icecream import ic

from streamlit_searchbox import st_searchbox


from decouple import config
elastic_cloud_id = config('ELASTIC_CLOUD_ID', default='none')
elastic_api_key = config('ELASTIC_API_KEY', default='none')
elastic_index_name = config('ELASTIC_INDEX_NAME', default='none')
raw_data = config('RAW_DATA', default='none')
elastic_synonym_fn = config('ELASTIC_SYNONYM_FILE', default='none')
elastic_synonym_id = config('ELASTIC_SYNONYM_ID', default='none')

In [21]:
elastic_client = Elasticsearch(
    cloud_id=elastic_cloud_id,
    api_key=elastic_api_key,
)

elastic_client.indices.delete(index="book_index", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

In [22]:
settings = {
    "analysis": {
        "analyzer": {
            "my_custom_index_analyzer": {
                "tokenizer": "standard",
                "filter": ["lowercase"],
            },
            "my_custom_search_analyzer": {
                "tokenizer": "standard",
                "filter": ["lowercase", "my_synonym_filter"],
            },
        },
        "filter": {
            "my_synonym_filter": {
                "type": "synonym_graph",
                "synonyms_set": "my-synonyms-set",
                "updateable": True,
            }
        },
    }
}

mappings = {
    "properties": {
        "title": {
            "type": "text",
            "analyzer": "my_custom_index_analyzer",
            "search_analyzer": "my_custom_search_analyzer",
        },
        "summary": {
            "type": "text",
            "analyzer": "my_custom_index_analyzer",
            "search_analyzer": "my_custom_search_analyzer",
        },
    }
}

elastic_client.indices.create(index="book_index", mappings=mappings, settings=settings)

ConnectionTimeout: Connection timed out